In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import re
import random
from random import randint
import time #delay
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import Select
from tqdm import tqdm
 
print(os.getcwd())

c:\Users\maimranj\Desktop\BIMB\backend\legisltaion\amendment act


In [2]:
driver=webdriver.Edge()

In [3]:
driver=webdriver.Edge()

act_no = []
title = []
date_of_royal_assent = []
publication_date = []
date_of_commencement = []
bm_link = []
en_link = []

url = 'https://lom.agc.gov.my/principal.php?type=amendment'
driver.get(url)
time.sleep(random.randint(5, 10))

# Loop through all 36 pages
for page in range(1, 37):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    amendment_act_list = soup.find('table', attrs={'class': 'table table-striped table-bordered dataTable'}).find_all('tr')[1:]

    for amendment_act in amendment_act_list:
        cols = amendment_act.find_all('td')

        if len(cols) >= 5:
            act_no.append(cols[0].get_text(strip=True))
            titles = cols[1].find_all('a')
            if len(titles) >= 2:
                title.append(f"{titles[0].get_text(strip=True)}\n{titles[1].get_text(strip=True)}")
            else:
                title.append(titles[0].get_text(strip=True))

            date_of_royal_assent.append(cols[2].get_text(strip=True))
            publication_date.append(cols[3].get_text(strip=True))
            date_of_commencement.append(cols[4].get_text(strip=True))

            pdf_links = cols[5].find_all('a', href=True)
            bm_pdf_link = next((link['href'] for link in pdf_links if "BM" in link['href'] and link['href'].endswith(".pdf")), 'N/A')
            en_pdf_link = next((link['href'] for link in pdf_links if "BI" in link['href'] and link['href'].endswith(".pdf")), 'N/A')

            bm_link.append(f"https://lom.agc.gov.my{bm_pdf_link}" if bm_pdf_link != 'N/A' else 'N/A')
            en_link.append(f"https://lom.agc.gov.my{en_pdf_link}" if en_pdf_link != 'N/A' else 'N/A')

    if page < 36:
        select_element = Select(driver.find_element(By.CSS_SELECTOR, 'div.dataTables_paginate select'))
        select_element.select_by_value(str(page + 1))
        time.sleep(random.randint(5, 10))

amend_df = pd.DataFrame({
    'Act No': act_no,
    'Title': title,
    'Date of Royal Assent': date_of_royal_assent,
    'Publication Date': publication_date,
    'Date of Commencement': date_of_commencement,
    'BM PDF Link': bm_link,
    'EN PDF Link': en_link
})

print(amend_df)

    Act No                                              Title  \
0    A1739  AKTA BEKALAN ELEKTRIK (PINDAAN) 2024\nELECTRIC...   
1    A1738  AKTA PEMBAYARAN DAN ADJUDIKASI INDUSTRI \nPEMB...   
2    A1737  AKTA TIMBANG TARA (PINDAAN) 2024\nARBITRATION ...   
3    A1736  AKTA KESALAHAN KESELAMATAN \n(LANGKAH-LANGKAH ...   
4    A1735  AKTA ANGKATAN TENTERA (PINDAAN) 2024\nARMED FO...   
..     ...                                                ...   
349  A1396  AKTA PERBEKALAN TAMBAHAN (2011) 2011\nSUPPLEME...   
350  A1395  AKTA TIMBANG TARA (PINDAAN) 2011\nARBITRATION ...   
351  A1394  AKTA PERKAPALAN SAUDAGAR (PENCEMARAN MINYAK) (...   
352  A1393  AKTA PERKAPALAN SAUDAGAR (PINDAAN DAN PERLUASA...   
353  A1392  AKTA PERBEKALAN TAMBAHAN (2010) 2011\nSUPPLEME...   

    Date of Royal Assent Publication Date  \
0             02/07/2024       26/11/2024   
1             23/10/2024       01/11/2024   
2             23/10/2024       01/11/2024   
3             23/10/2024       01/11/20

In [4]:
driver.close()

In [6]:
amend_df.to_csv('amendment_act.csv')